In [1]:
# === Put-it-to-the-test (Colab single cell) ===
# 1) A-invariant check; 2) Near-onset Lyapunov scaling for two unimodal maps.

import math, numpy as np, pandas as pd

# ---------- 1) A-invariant ----------
delta = 4.669201609102990     # Feigenbaum delta
e = math.e
A = (1 - e/delta) / math.sqrt(delta/e)
print("A-invariant candidate:")
print(f"  A = (1 - e/delta)/sqrt(delta/e) = {A:.12f}")
print(f"  1/pi = {1/math.pi:.12f}")
print(f"  A - 1/pi = {A - 1/math.pi:.12e}")
print(f"  relative error vs 1/pi = {(A - 1/math.pi)/(1/math.pi):.3e}\n")

# ---------- 2) Near-onset Lyapunov scaling ----------
# Map 1: Logistic  x_{n+1} = r x (1-x)
r_inf = 3.569945672
def f_logistic(x, r): return r * x * (1 - x)
def df_logistic(x, r): return r * (1 - 2*x)

# Map 2: Quadratic  x_{n+1} = 1 - a x^2   (on [-1,1])
a_inf = 1.401155189
def f_quad(x, a): return 1 - a * x * x
def df_quad(x, a): return -2 * a * x

def lyapunov(map_f, dmap_f, param, x0, n_trans=4000, n_iter=15000):
    x = x0
    for _ in range(n_trans):
        x = map_f(x, param)
    s = 0.0
    for _ in range(n_iter):
        d = dmap_f(x, param)
        if d == 0.0:
            d = 1e-300
        s += math.log(abs(d))
        x = map_f(x, param)
    return s / n_iter

def fit_linear_log_inv_eps(eps, lamb):
    x = np.log(1.0/np.array(eps))
    y = np.array(lamb)
    X = np.vstack([x, np.ones_like(x)]).T
    m,b = np.linalg.lstsq(X, y, rcond=None)[0]
    yhat = m*x + b
    ss_res = np.sum((y - yhat)**2)
    ss_tot = np.sum((y - np.mean(y))**2)
    R2 = 1 - ss_res/ss_tot if ss_tot>0 else float('nan')
    return m,b,R2

# eps grid (very small, just above onset)
eps_values = np.logspace(-6.5, -4.2, 12)  # ~3e-7 to ~6e-5

rows = []
# logistic
for eps in eps_values:
    lam = lyapunov(f_logistic, df_logistic, r_inf+eps, x0=0.321)
    rows.append({"map":"logistic","eps":float(eps),"lambda":float(lam)})
# quadratic
for eps in eps_values:
    lam = lyapunov(f_quad, df_quad, a_inf+eps, x0=0.123)
    rows.append({"map":"quadratic","eps":float(eps),"lambda":float(lam)})

df = pd.DataFrame(rows)

# Fit on the smallest 8 eps (closest to onset)
for name in ["logistic","quadratic"]:
    sub = df[df["map"]==name].sort_values("eps").head(8)
    m,b,R2 = fit_linear_log_inv_eps(sub["eps"].values, sub["lambda"].values)
    print(f"{name:>9}: lambda ~ m*log(1/eps) + b     m={m:.6f}   R^2={R2:.4f}")
print("\n(If both slopes m are similar and R^2 is high, it supports a shared near-onset law across maps.)")


A-invariant candidate:
  A = (1 - e/delta)/sqrt(delta/e) = 0.318803230318
  1/pi = 0.318309886184
  A - 1/pi = 4.933441343236e-04
  relative error vs 1/pi = 1.550e-03

 logistic: lambda ~ m*log(1/eps) + b     m=-0.000216   R^2=0.0216
quadratic: lambda ~ m*log(1/eps) + b     m=-0.000975   R^2=0.9346

(If both slopes m are similar and R^2 is high, it supports a shared near-onset law across maps.)
